In [8]:
import pandas as pd
import numpy as np
import re
import math
import tabula
import camelot
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO 
import matplotlib.pyplot as plt 

In [50]:
def Sbi_tab(f):
    try:
        tables=tabula.read_pdf(f,
                                  lattice=True,
                                  pages="all",
                                  silent=True,
                                  multiple_tables=True,
                                  pandas_options={'header':None})
        df = pd.DataFrame()
        df = pd.concat([c for c in tables]).drop_duplicates()
    except: pass
    return df

def Sbi_process(df):
    df = df.replace(r'^\s*$', np.nan, regex=True)
    try:
        idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('date').any(),axis=1)==True].index][0]
        df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
    except:
        try:
            idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('description').any(),axis=1)==True].index][0]
            df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
        except:
            print("\nSBI Column Headers Missing"); pass
        try:
            df = df.drop(["Init."], axis=1)
        except:
            try:
                df = df.drop(["Branch Name"], axis=1)
            except:pass

    df = df[~df.index.isin(df[df.apply(lambda row:row.astype(str).str.lower().str.contains('opening balance|transaction total|closing balance').any(),axis=1)==True].index)]
    #df = pd.DataFrame(df.T.drop_duplicates().T)
    df.drop(df.nunique(dropna=False)[(df.nunique(dropna=False) == 1)].index, axis=1, inplace=True)

    df.to_csv("parsed.csv",index=0)
    return df

f = r"C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\SBI\files\sbi31.pdf"
try:
    df = Sbi_tab(f) ; df = Sbi_process(df)
    print("Parsed")
except:
    print("Not Parsed")

Parsed
